## Compression de Lempel-Ziv-Welsh

Valentin Chomel - 01/02/2019

In [70]:
#Imports
import sys
import os
from sys import argv
from struct import *

#### Début de la compression : 

Lecture dans le fichier texte

In [71]:
fichier = open("texte.txt", "r")
input_file = fichier.read()

n = len(input_file)  
            
maximum_table_size = pow(2,int(n))                           

Affichage des données du fichier initial : 

In [72]:
#print(input_file)

Construction et initialisation du dictionnaire :

In [73]:
# Initialisation du dictionnaire 
dictionary_size = 256                   
dictionary = {chr(i): i for i in range(dictionary_size)}    
string = ""             # String is null.
compressed_data = []    # variable to store the compressed data.

##### Algorithme de compression : 

FONCTION LZW_Compresser(Texte, dictionnaire)

    w ← ""

    TANT QUE (il reste des caractères à lire dans Texte) FAIRE
       c ← Lire(Texte)
       p ← Concaténer(w, c)
       SI Existe(p, dictionnaire) ALORS
          w ← p
       SINON
          Ajouter(p, dictionnaire)
          Écrire dictionnaire[w]
          w ← c
    FIN TANT QUE

Implementation de l'algorithme :

In [74]:
#Implementation de l'algo : 
for symbol in input_file:                     
    string_plus_symbol = string + symbol # get input symbol.
    if string_plus_symbol in dictionary: 
        string = string_plus_symbol
    else:
        compressed_data.append(dictionary[string])
        if(len(dictionary) <= maximum_table_size):
            dictionary[string_plus_symbol] = dictionary_size
            dictionary_size += 1
        string = symbol

if string in dictionary:
    compressed_data.append(dictionary[string])

Affichage des données compressées pour vérification :

In [75]:
#print(compressed_data)

Enregistrement des données dans un fichier (binaire) : 

In [76]:
output_file = open("./files/texte_encoded.txt", "wb")
for data in compressed_data:
    output_file.write(pack('>H',int(data)))
    
output_file.close()
fichier.close()

#### Fin de la compression

#### Début de la décompression 

Ouverture de fichier des données compressées : 

In [77]:
file = open('./files/texte_encoded.txt', "rb")
compressed_data = []
next_code = 256
decompressed_data = ""
string = ""

Lecture des données compressées :

In [78]:
while True:
    rec = file.read(2)
    if len(rec) != 2:
        break
    (data, ) = unpack('>H', rec)
    compressed_data.append(data)

Contruction et initialisation du dictionnaire : 

In [79]:
dictionary_size = 256
dictionary = dict([(x, chr(x)) for x in range(dictionary_size)])

##### Algorithme de décompression :

FONCTION LZW_Décompresser(Code, dictionnaire)

     n ← |Code|
     v ← Lire(Code)
     Écrire dictionnaire[v]
     w ← chr(v)

     POUR i ALLANT DE 2 à n FAIRE
        v ← Lire(Code)
        SI Existe(v, dictionnaire) ALORS
          entrée ← dictionnaire[v]
        SINON entrée ← Concaténer(w, w[0])
        Écrire entrée
        Ajouter(Concaténer(w,entrée[0]), dictionnaire)
        w ← entrée
     FIN POUR

Application de l'agorithme :

In [80]:
#Application de l'algo
for code in compressed_data:
    if not(code in dictionary):
        dictionary[code] = string + (string[0])
    decompressed_data += dictionary[code]
    if not(len(string)==0):
        dictionary[next_code] = string + (dictionary[code][0])
        next_code += 1
    string = dictionary[code]

Affichage des données décompressées : 

In [81]:
#print(decompressed_data)

Enregistrement des données décompressées dans un fichier :

In [82]:
#Enregistrement des données décompressées dans un fichier
output_file = open("./files/texte_decoded.txt", "w")
for data in decompressed_data:
    output_file.write(data)
output_file.close()
file.close()

#### Fin de la décompression. 

#### Résultats : 

Affichage des resultats de la compression Lempel-Ziv-Welsh :

In [83]:
#Affichage des resultats de la compression Lempel-Ziv-Welsh : 
size_txt = os.path.getsize('./texte.txt')
size_txt_encoded = os.path.getsize('./files/texte_encoded.txt')
size_txt_decoded = os.path.getsize('./files/texte_decoded.txt')

print("La taille du texte initial est de :", size_txt, "ko \n")
print("La taille du texte compressé est de :", size_txt_encoded, "ko \n")
print("La taille du texte décompressé est de :", size_txt_decoded, "ko\n")

La taille du texte initial est de : 197664 ko 

La taille du texte compressé est de : 89012 ko 

La taille du texte décompressé est de : 197664 ko

